In [303]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
from bisect import bisect

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

First, we want to write a function that scrapes the data from the ny mta website.  The url for the pages containing the data end with a *week-number* specifying the week when the data was taken.  This number is a 6-digit number like *180623*, which means *June 23, 2018*.  Starting from a given known week, we want to go back for a specified number of years, collecting data from a specific month.

To do this, we need to first write a small function which returns the list of week- numbers in the appropriate format.  The function will take 'month' and 'yrs_back' as arguments and then return a list of week-numbers going 'yrs_back' number of years back.  For instance, **get_weeks_nums(3,2)** gives all the weeks in march for two years back, starting from the most recent dataset.  The function **fix_time** just helps a bit with formatting.

Finally, we want to scrape the data from the website by looking up all the url's with the appropriate week-numbers.  This is accomplished in the code below.

In [304]:
def fix_time(num):
    if len(str(num)) == 2:
        return str(num)
    else:
        return '0'+str(num)

def get_week_nums(month,yrs_back):
    week_list=[]
    ref_date=datetime.date(2018,6,30)
    weeks_back=yrs_back*52
    for i in range(weeks_back):
        week_shift=datetime.timedelta(-7*i)
        new=ref_date+week_shift
        yr=str(new.year)[-2:]
        mt=fix_time(new.month)
        day=fix_time(new.day)
        string=yr+mt+day
        if int(mt)==month:
            week_list.append(int(string))
    return week_list

def scrape(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

#I am going to focus on June for the last 3 years.
week_nums = get_week_nums(6,3)
df = scrape(week_nums)

It is convenient to strip off the whitespace around the column names.  The name *df_small* is poorly chosen, but I'm too lazy to change it.

In [306]:
cols={x:x.strip() for x in df.columns}
df_small=df.rename(columns=cols)

Next we want to extract a time stampe from a given row in the dataframe.  The following is an example of how to do this for a particular row.  Once we know how to do this on a given row we can write a function which takes in a row and returns a timestamp.  We can then apply this function to the dataframe and add a new column with the 'datetime' information.

In [326]:
pd.to_datetime(df_small.iloc[1][6]+' '+df_small.iloc[1][7],infer_datetime_format=True)

Timestamp('2018-06-23 04:00:00')

In [307]:
def get_datetime(x):
    return pd.to_datetime(x[6]+' '+x[7],infer_datetime_format=True)

In [308]:
df_small['datetime']=df_small.apply(get_datetime,axis=1)

Next we do some cleaning.  The function 'fn' allows us to get rid of elements which don't have zero seconds.  These appear to be spurious. (Perhaps these represent cases where the turnstile data was recorded mid-period for some reason.) Since the data is cumulative, removing a row will not affect subsequent calculations.  We will also drop duplicate information.  From visual inspection, the duplicates appear to be situations where the data was lost, recovered, and re-found.

In [309]:
def fn(row):
    return row['datetime'].second==0

df_small_clean=df_small[df_small.apply(fn,axis=1)].drop_duplicates(subset=['C/A','UNIT','SCP','STATION','LINENAME','datetime'])

The next steps, clean2 and clean3 allow us to focus on the relevant information, i.e., data at the station and time level.  We really don't care about the particular turnstile.  Looking at the particular turnstile could be useful in principle, but, in practice, the person handing out fliers will just go to the specified station and then figure out where the people are.  In the worse case, he/she might have to walk across the street.  However, the data will not be as useful as what the solictor sees on the ground and so we'll leave this to them.

In [310]:
df_small_clean2=df_small_clean.groupby(['STATION','datetime'])[['EXITS']].sum()
df_small_clean3=df_small_clean2.reset_index()

Now we need to get diffs for each station, which means we need to find a list of stations, go by each station, sort by datetime, then do a diff operation, etc.  

As a first step, we want a function which inputs a dataframe and station and then gets only the data relevant to that station.  The function will also sort by datetime for later convenience.

In [311]:
def station_activity(df,station):
    df_station=df[df['STATION']==station]
    df_sort=df_station.sort_values(by=['datetime'])
    return df_sort

Below is an example of what this function does on dataframes formatted like df_small_clean3.

In [85]:
station_activity(df_small_clean3,'59 ST').head()

,STATION,datetime,EXITS
58360,59 ST,2016-05-28 00:00:00,955460529
58361,59 ST,2016-05-28 04:00:00,955461609
58362,59 ST,2016-05-28 08:00:00,955464213
58363,59 ST,2016-05-28 12:00:00,955471806
58364,59 ST,2016-05-28 16:00:00,955480958


Next, go by station and for each station, apply the diff operation, obtaining a list of dataframes containing the diff values.  The list is actually stored as a dictionary, 'station_diffs_dct' since we want to call the entries by the station names rather than a number.

In [327]:
stations=df_small_clean3['STATION'].unique()
station_diffs_dct={}
for st in stations:
    sa=station_activity(df_small_clean3,st)
    sa['diffs']=pd.DataFrame(sa['EXITS'].diff())
    st_diffs=sa.drop(['EXITS'],axis=1).dropna()
    st_diffs_clean=st_diffs[np.abs(st_diffs.diffs)<10**6]    
    st_diffs_clean['weekday']=st_diffs_clean['datetime'].apply(lambda x: x.weekday())
    st_diffs_clean['hour']=st_diffs_clean['datetime'].apply(lambda x: x.hour)
    st_diffs_clean2=st_diffs_clean.groupby(['weekday','hour'])['diffs'].mean()
    station_diffs_dct[st]=st_diffs_clean2

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


And here is an example of what this does:

In [362]:
station_diffs_dct['59 ST']

weekday  hour
0        0        2638.454545
         4         552.076923
         8        7175.666667
         12      18147.636364
         16       9420.000000
         20       9763.153846
1        0        2931.583333
         4         538.538462
         8        8619.166667
         12      22129.000000
         16      10903.583333
         20      11548.230769
2        0        3706.384615
         4         580.769231
         8        8597.833333
         12      22243.083333
         16      11222.230769
         20      11751.538462
3        0        3873.833333
         4         612.666667
         8        8522.846154
         12      21888.083333
         16      10977.461538
         20      11228.230769
4        0        3690.307692
         4         645.846154
         8        7772.166667
         12      19652.666667
         16      10803.384615
         20      10285.307692
5        0        3948.000000
         4        1055.769231
         8        2536.615

Finally, we want a function which allows us to input a day of the week and a time and then return a list of stations.  It is a little bit problematic that different stations have data collected on different schedules.  No problem.  We just need to wite a function which takes the collection hours of a given station and then, for the hour input by the user, figures out when the next appropriate hour for that station is.  The function **find_key** below does this.  Our final function **activity_by_time** provides a list for the desired day + time.  

In [363]:
def find_key(num,hours_list):
    sort=sorted(hours_list)
    pos=bisect(sort,num)
    if pos < len(sort):
        return sort[pos]
    else:
        return sort[0]
    
def activity_by_time(day,hour):
    #We must convert a 24hour of the day to one of 0,4,8...
    exits=[]
    for st in stations:
        try:
            #hours_st=[x.hour for x in station_diffs_dct[st]['datetime'][:6]]
            hours_st=station_diffs_dct[st][0].keys().values
            sh=sorted(hours_st)
            h_key=find_key(hour,sh)
            leaving=station_diffs_dct[st][day][h_key]/4
            exits.append([st,leaving])
        except(KeyError,IndexError,AttributeError):
            pass
    sort_exits=sorted(exits,key=lambda x: x[1])[::-1]    
    return sort_exits

Now we are done.  When inputting the time remember that Monday is zero, Tuesday is 1, etc.  The hour may be input as an integer and it internally converts to the appropriate time interval for the given station using the find_key function.  Below is an example.

In [360]:
activity_by_time(3,20)[:10]

[['JEFFERSON ST', 8948.403846153846],
 ['14 ST-UNION SQ', 5806.954545454545],
 ['JOURNAL SQUARE', 3806.1666666666665],
 ['W 4 ST-WASH SQ', 2929.653846153846],
 ['7 AV', 2355.5625],
 ['34 ST-HERALD SQ', 2351.6346153846152],
 ['72 ST', 2345.4375],
 ['FLUSHING-MAIN', 2254.923076923077],
 ["B'WAY-LAFAYETTE", 2012.2045454545455],
 ['KEW GARDENS', 1969.3958333333333]]